In [3]:
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 處理找不到元素的工具
from selenium.common.exceptions import NoSuchElementException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 加入行為鍊 ActionChain (在 WebDriver 中模擬滑鼠移動、點繫、拖曳、按右鍵出現選單，以及鍵盤輸入文字、按下鍵盤上的按鈕等)
from selenium.webdriver.common.action_chains import ActionChains

# 加入鍵盤功能 (例如 Ctrl、Alt 等)
from selenium.webdriver.common.keys import Keys

# 強制等待 (執行期間休息一下)
import time
from time import sleep

# 匯入 regex 套件
import re

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import pprint

# 平行任務處理
from concurrent.futures import ThreadPoolExecutor as tpe

# 使用CSV檔
import csv

# 使用pandas
import pandas as pd

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")  #不開啟實體瀏覽器背景執行
# my_options.add_argument("--start-maximized")  #最大化視窗
my_options.add_argument("--incognito")  #開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文


# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

In [4]:
nameList = []
pf1 = pd.read_json('cityroad/newTaipeiCityRoad.json')
df1 = pd.DataFrame(pf1)
road = df1[0][30:] #11/30 30+21

# 輸入名稱
# for j in road :
#     driver.get(f'https://www.google.com.tw/maps/search/新北市{j} 飲料店/@23.546162,120.6402133,8z?hl=zh-TW%27')
#     sleep(90)
#  web=driver.find_elements(By.CSS_SELECTOR, 'div[data-js-log-root] div[role="article"] > div[aria-label]')
#    for a in web:
#    print(a.get_attribute("href"))
    
for j in road :
    driver.get(f'https://www.google.com.tw/maps/search/新北市 {j} 飲料店/@23.546162,120.6402133,8z?hl=zh-TW%27')
    sleep(2)
        
    offset = 0
    innerHeight = 0
    count = 0  # 累計無效滾動次數
    limit = 2  # 最大無效滾動次數

    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
    # 等待篩選元素出現
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div[role="feed"]')
            )
        )

        # focus: 主角頁面
        focus = driver.find_element(By.CSS_SELECTOR, 'div[role="feed"]')

        # offset: 拉槓到頁面頂端的距離
        offset = driver.execute_script('return arguments[0].scrollTop', focus)
        # print(offset)

        # 執行js指令捲動頁面
        driver.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight)', focus)

        # innerHeight: 頁面高度 = 拉槓到頁面頂端的距離
        innerHeight = driver.execute_script('return arguments[0].scrollHeight = arguments[0].scrollTop', focus)
        # print(innerHeight)

        sleep(2)  # 依網路狀況調整

        # 經過計算，如果「拉槓到頁面頂端的距離」(offset)等於「頁面高度 = 拉槓到頁面頂端的距離」(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1

        # 計數器等於限制數則跳脫
        if count == limit:
            break    

    print("搜尋結束 抓資料")
    # 抓飲料店在google map上的網址
    for a in driver.find_elements(By.CSS_SELECTOR,'div[data-js-log-root] div[role="article"] a[aria-label]'):
        nameList.append(a.get_attribute('href'))        
    sleep(2)
    
    # 抓飲料店資料
    dataList = []
    for i in range(len(nameList)):
        try:
            driver.get(nameList[i])

            try:
                place_name = driver.find_element(By.CSS_SELECTOR,'div[class="TIHn2"] h1 span').get_attribute('innerText')
            except NoSuchElementException:
                place_name = ""
            pass

            sleep(3)

            try:
                address = driver.find_element(By.CSS_SELECTOR,'div[data-js-log-root] div[class="rogA2c"] div[class="Io6YTe fontBodyMedium"]').get_attribute('innerText')
            except NoSuchElementException:
                address = ""
            pass

            try:
                place_category = driver.find_element(By.CSS_SELECTOR, 'div[data-js-log-root] div[data-js-log-root] div[style^="font-family"] > span button[jsaction][jstcache][class][jsan]').get_attribute('innerText')
            except NoSuchElementException:
                place_category = ""
            pass

            if address != "":
                dataList.append({
                    'address' : address,
                    'place_name' : place_name,
                    'place_category' : place_category
                })

        except: 
            continue
            
    import json, os, pprint, time, re
    # 沒有放置 txt 檔的資料夾，就建立起來
    folderPath = '新北市飲料店'
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    with open(f"{folderPath}/新北市飲料店2.json", "w", encoding="utf-8") as file:
            file.write( json.dumps(dataList, ensure_ascii=False) )

搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料
搜尋結束 抓資料


TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00BEACD3+2075859]
	Ordinal0 [0x00B7EE61+1633889]
	Ordinal0 [0x00A7B7BD+571325]
	Ordinal0 [0x00AAAC2F+764975]
	Ordinal0 [0x00AAAE1B+765467]
	Ordinal0 [0x00ADD0F2+970994]
	Ordinal0 [0x00AC7364+881508]
	Ordinal0 [0x00ADB56A+963946]
	Ordinal0 [0x00AC7136+880950]
	Ordinal0 [0x00A9FEFD+720637]
	Ordinal0 [0x00AA0F3F+724799]
	GetHandleVerifier [0x00E9EED2+2769538]
	GetHandleVerifier [0x00E90D95+2711877]
	GetHandleVerifier [0x00C7A03A+521194]
	GetHandleVerifier [0x00C78DA0+516432]
	Ordinal0 [0x00B8682C+1665068]
	Ordinal0 [0x00B8B128+1683752]
	Ordinal0 [0x00B8B215+1683989]
	Ordinal0 [0x00B96484+1729668]
	BaseThreadInitThunk [0x76A4FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77277BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77277B8E+238]


In [1]:
nameList = []
pf1 = pd.read_json('cityroad/newTaipeiCityRoad.json')
df1 = pd.DataFrame(pf1)
road = df1[0][11:]
road
# def Data():
#     dataList = []
    
#     for i in range(len(nameList)):
        
#         driver.get(nameList[i])
        
#         try:
#             place_name = driver.find_element(By.CSS_SELECTOR,'div[class="TIHn2"] h1 span').get_attribute('innerText')
#         except NoSuchElementException:
#             place_name = ""
#         pass
        
#         sleep(3)
    
#         try:
#             address = driver.find_element(By.CSS_SELECTOR,'div[data-js-log-root] div[class="rogA2c"] div[class="Io6YTe fontBodyMedium"]').get_attribute('innerText')
#         except NoSuchElementException:
#             address = ""
#         pass
        
#         #try:
#         #    stytle = driver.find_element(By.CSS_SELECTOR,'div[class="bfdHYd Ppzolf OFBs3e"] div[class="lI9IFe"] div[class="y7PRA"] div[class="W4Efsd"] span').get_attribute('innerText')
#         #except NoSuchElementException:
#         #    stytle = ""
#         #pass
    
#         try:
#             place_category = driver.find_element(By.CSS_SELECTOR, 'div[data-js-log-root] div[data-js-log-root] div[style^="font-family"] > span button[jsaction][jstcache][class][jsan]').get_attribute('innerText')
#         except NoSuchElementException:
#             place_category = ""
#         pass
 

#         dataList.append({
#             'address' : address,
#             'place_name' : place_name,
#             'place_category' : place_category
#         })
    

#         #pprint.pprint(dataList)
#         with open("飲料店.json", "w", encoding="utf-8") as file:
#             file.write( json.dumps(dataList, ensure_ascii=False) )

NameError: name 'pd' is not defined